In [3]:
import os
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import datetime as dt
import re

import logging
logging.basicConfig(level=logging.INFO)

import altair as alt
alt.data_transformers.disable_max_rows();

from src.data.query_snowflake import update_week_extremes, read_DALI_week_extremes, read_meta

In [4]:
update_week_extremes()

In [ ]:
sample_box = "ESD.000400-1"

In [ ]:
%%time 
df_data = read_DALI_week_extremes(boxid=sample_box, L="sumli")
df_data

In [ ]:
%%time
df_meta = read_meta(sample_box)
df_meta

In [ ]:
df_plot = df_data.copy()
df_plot["date"] = df_plot.apply(lambda df: dt.datetime.fromisocalendar(df["year"], df["week"] , 1), axis=1)
value_vars = ["min", "max"]
df_plot = df_plot.melt(
    var_name="history",
    value_name="power",
    id_vars=df_plot.columns.difference(value_vars),
    value_vars=value_vars,
)

In [ ]:
def plot_observed(df):

    alt_observed = (
        alt.Chart(df)
        .mark_point(color="black")
        .encode(
            x=alt.X("date:T", title="date"),
            y=alt.Y("power:Q", title="power [kW]"),
            shape=alt.Shape(
                "history",
                scale=alt.Scale(
                    domain=["max", "min"], range=["triangle-up", "triangle-down"],
                ),
            ),
            tooltip=[
                alt.Tooltip("power:Q", format=".2f"),
                alt.Tooltip("history:N"),
                alt.Tooltip("year:Q"),
                alt.Tooltip("week:Q"),
            ],
        )
        .properties(width=800)
    ).interactive()
    return alt_observed

In [ ]:
# df_data = read_DALI_week_extremes(boxid=sample_box, L="sumli")
# df_data["date"] = df_data.apply(lambda df: dt.datetime.fromisocalendar(df["year"], df["week"] , 1), axis=1)

In [ ]:
# df_meta.sample()

In [ ]:
# df_meta[df_meta["boxid"]==sample_box]

In [ ]:
# df_data["date"] = df_data.apply(lambda df: dt.datetime.fromisocalendar(df["year"], df["week"] , 1), axis=1)

In [ ]:
def plot_observed(df):

    alt_observed = (
        alt.Chart(df)
        .mark_point(color="black")
        .encode(
            x=alt.X("date:T", title="date"),
            y=alt.Y("power:Q", title="power [kW]"),
            shape=alt.Shape(
                "history",
                scale=alt.Scale(
                    domain=["max", "min"],
                    range=["triangle-up", "triangle-down"],
                ),
            ),
            tooltip=[
                alt.Tooltip("power:Q", format=".2f"),
                alt.Tooltip("history:N"),
                alt.Tooltip("year:Q"),
                alt.Tooltip("week:Q"),
            ],
        )
        .properties(width=800)
    ).interactive()
    return alt_observed


def plot_limits(df_limits):
    df_limits["transformer"] = "capacity"
    if "date" in df_limits.columns:
        return (
            alt.Chart(df_limits)
            .mark_line(strokeDash=[5, 5])
            .encode(
                x=alt.X("date:T", title="date"),
                y=alt.Y("value:Q", title=""),
                color=alt.Color("transformer:N", scale=alt.Scale(range=["red"])),
            )
            .properties(width=800)
        ).interactive()
    else:
        return (
            alt.Chart(df_limits)
            .mark_rule(strokeDash=[5, 5])
            .encode(
                y=alt.Y("value:Q", title=""),
                color=alt.Color(
                    "transformer:N", title="", scale=alt.Scale(range=["red"])
                ),
            )
            .properties(width=800)
        ).interactive()

In [ ]:
def format_limits(df_meta, df_data=None):
    limit = df_meta["vermogen_nominaal"].squeeze()

    if df_data is None:
        return pd.DataFrame({"limit": ["lower", "upper"], "value": [-limit, limit]})
    else:
        timestamps = [df_data["date"].min(), df_data["date"].max()]

        return pd.DataFrame(
            {
                "date": timestamps,
                "lower": [-limit] * len(timestamps),
                "upper": [limit] * len(timestamps),
            }
        ).melt(id_vars=["date"], value_vars=["lower", "upper"], var_name=["limit"])

In [ ]:
df_limits = format_limits(df_meta)

In [ ]:
alt.layer(plot_limits(df_limits), plot_observed(df_plot))

In [ ]:
plot_limits(df_limits) + plot_observed(df_plot)